#Recommendation engine


In [5]:
import pandas as pd 
import numpy as np
import warnings
import gc
import datetime

In [6]:
df = pd.read_csv(r'D:\recommendation engine\test_J1hm2KQ.csv', encoding = 'ISO-8859-1')

In [7]:
df.head()

,Country,CustomerID,InvoiceDate,InvoiceNo,Quantity,StockCode,UnitPrice
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [8]:
df.rename(index=str, columns={'InvoiceNo': 'invoice_num',
                              'StockCode' : 'stock_code',
                              'Quantity' : 'quantity',
                              'InvoiceDate' : 'invoice_date',
                              'UnitPrice' : 'unit_price',
                              'CustomerID' : 'cust_id',
                              'Country' : 'country'}, inplace=True)

In [9]:
df.head()

,country,cust_id,invoice_date,invoice_num,quantity,stock_code,unit_price
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [10]:
#data Cleaning
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103097 entries, 0 to 103096
Data columns (total 7 columns):
country         103097 non-null object
cust_id         103097 non-null int64
invoice_date    103097 non-null object
invoice_num     103097 non-null int64
quantity        103097 non-null int64
stock_code      103097 non-null object
unit_price      103097 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 6.3+ MB


In [11]:
# check missing values for each column 
df.isnull().sum().sort_values(ascending=False)

unit_price      0
stock_code      0
quantity        0
invoice_num     0
invoice_date    0
cust_id         0
country         0
dtype: int64

In [14]:
# change columns tyoe - String to Int type 
df['cust_id'] = df['cust_id'].astype('int64')

In [15]:
df.head()

,country,cust_id,invoice_date,invoice_num,quantity,stock_code,unit_price
0,PX,127269,01/12/10 8:28,127269,7,22633V,1.85
1,PX,227268,01/12/10 8:34,227268,38,84879M,1.69
2,PX,227268,01/12/10 8:34,227268,7,22748P,2.10
3,PX,227268,01/12/10 8:34,227268,9,22749K,3.75
4,PX,227268,01/12/10 8:34,227268,2,22622G,9.95


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 103097 entries, 0 to 103096
Data columns (total 7 columns):
country         103097 non-null object
cust_id         103097 non-null int64
invoice_date    103097 non-null object
invoice_num     103097 non-null int64
quantity        103097 non-null int64
stock_code      103097 non-null object
unit_price      103097 non-null float64
dtypes: float64(1), int64(3), object(3)
memory usage: 6.3+ MB


In [17]:
df.describe().round(2)

,cust_id,invoice_num,quantity,unit_price
count,103097.00,103097.00,103097.00,103097.00
mean,251697.53,251697.53,10.41,3.30
std,148216.58,148216.58,256.46,5.18
min,1890.00,1890.00,-80995.00,0.00
25%,130869.00,130869.00,1.00,1.25
50%,243747.00,243747.00,3.00,2.08
75%,364527.00,364527.00,12.00,4.13
max,579015.00,579015.00,5160.00,649.50


In [18]:
df_new = df[df.quantity > 0]

In [19]:
df_new.describe().round(2)

,cust_id,invoice_num,quantity,unit_price
count,101335.00,101335.00,101335.00,101335.00
mean,251638.08,251638.08,11.85,3.28
std,148220.04,148220.04,42.31,4.87
min,1890.00,1890.00,1.00,0.00
25%,130869.00,130869.00,1.00,1.25
50%,243648.00,243648.00,4.00,2.08
75%,364527.00,364527.00,13.00,4.13
max,579015.00,579015.00,5160.00,649.50


In [20]:
df_new['amount_spent'] = df_new['quantity'] * df_new['unit_price']

In [21]:
# rearrange all the columns for easy reference
df_new = df_new[['invoice_num','invoice_date','stock_code','quantity','unit_price','amount_spent','cust_id','country']]

In [22]:
#different order of customes,Amount spent by customers,numbers of orders from dec2010 to dec2011,
#number of orders in a day,number of orders per hour are in tableau


In [24]:
#Discover patterns for unit prices
df_new.unit_price.describe()

count    101335.000000
mean          3.280237
std           4.869873
min           0.000000
25%           1.250000
50%           2.080000
75%           4.130000
max         649.500000
Name: unit_price, dtype: float64

In [25]:
#there are unit price with value 0,which means they are given free
df_free = df_new[df_new.unit_price == 0]

In [26]:
df_free.head()

,invoice_num,invoice_date,stock_code,quantity,unit_price,amount_spent,cust_id,country
4910,305487,10/12/10 14:59,22734C,36,0.0,0.0,305487,PX
7612,44559,20/12/10 10:36,21479J,902,0.0,0.0,44559,PX
8910,267327,06/01/11 16:41,22090X,28,0.0,0.0,267327,PX
8911,267327,06/01/11 16:41,22553Q,28,0.0,0.0,267327,PX
9701,164628,10/01/11 16:04,21644N,172,0.0,0.0,164628,PX


In [31]:
df_free.count()

invoice_num     74
invoice_date    74
stock_code      74
quantity        74
unit_price      74
amount_spent    74
cust_id         74
country         74
dtype: int64

In [ ]:
#number of free items given in the month is in tab

In [32]:
df_new.head()

,invoice_num,invoice_date,stock_code,quantity,unit_price,amount_spent,cust_id,country
0,127269,01/12/10 8:28,22633V,7,1.85,12.95,127269,PX
1,227268,01/12/10 8:34,84879M,38,1.69,64.22,227268,PX
2,227268,01/12/10 8:34,22748P,7,2.10,14.70,227268,PX
3,227268,01/12/10 8:34,22749K,9,3.75,33.75,227268,PX
4,227268,01/12/10 8:34,22622G,2,9.95,19.90,227268,PX


In [ ]:
#orders for each country in tab